In [3]:
import pandas as pd
import os

# Google

## Extraccion de locales

In [20]:
reviews=pd.read_csv('../datasets/csv/G_ulta_beauty.csv')

### Lectura de datos

In [21]:
# Especifica la ruta de la carpeta que contiene los archivos .parquet particionados
ruta_carpeta_parquet = '../datasets/parquets/metadatos Google'

# Lista para almacenar DataFrames de cada partición
dataframes = []

# Itera sobre los archivos .parquet en la carpeta
for root, dirs, files in os.walk(ruta_carpeta_parquet):
    for file in files:
        if file.endswith('.parquet'):
            # Construye la ruta completa del archivo
            ruta_completa = os.path.join(root, file)
            
            # Lee el archivo .parquet y agrega el DataFrame a la lista
            df_particion = pd.read_parquet(ruta_completa)
            dataframes.append(df_particion)

# Concatena todos los DataFrames en uno solo
df_google = pd.concat(dataframes, ignore_index=True)

### Extraccion

In [22]:
# Filtrar por el valor 'ulta beauty' en la columna 'name'
df_google = df_google[df_google['name'] == 'Ulta Beauty']
# Eliminamos columnas innecesarias
df_google = df_google[["gmap_id",'url',"latitude","longitude","avg_rating","State", "state", "address"]]
# Eliminamos Duplicados
df_google.drop_duplicates(inplace=True)
# Se exporta
df_google.to_csv('../datasets/csv/G_ulta_establishments.csv', index=False)

# Scrapping

In [ ]:
#Instalamos extension para coneccion
pip install apify-client

### Importar API Y datos

In [4]:
from apify_client import ApifyClient

apify_client = ApifyClient('apify_api_USgUEX5UbJ4Z0YMJoKF4S1MYt9mCmw4mmrzs')

# Fetch results from the actor's default dataset
dataset = apify_client.dataset('IwxKoLSghgggI6jm1')

#Extraigo los datos
dataset_items=dataset.list_items()

#Los paso a dataframe
data_API=pd.DataFrame(data=dataset_items.items)

In [5]:
data_API.columns

Index(['url', 'searchString', 'rank', 'searchPageUrl', 'searchPageLoadedUrl',
       'isAdvertisement', 'title', 'subTitle', 'description', 'price',
       'categoryName', 'address', 'neighborhood', 'street', 'city',
       'postalCode', 'state', 'countryCode', 'website', 'phone',
       'phoneUnformatted', 'claimThisBusiness', 'location', 'locatedIn',
       'plusCode', 'menu', 'totalScore', 'permanentlyClosed',
       'temporarilyClosed', 'placeId', 'categories', 'cid', 'reviewsCount',
       'reviewsDistribution', 'imagesCount', 'imageCategories', 'scrapedAt',
       'reserveTableUrl', 'googleFoodUrl', 'hotelStars', 'hotelDescription',
       'checkInDate', 'checkOutDate', 'similarHotelsNearby',
       'hotelReviewSummary', 'hotelAds', 'popularTimesLiveText',
       'popularTimesLivePercent', 'popularTimesHistogram', 'openingHours',
       'peopleAlsoSearch', 'placesTags', 'reviewsTags', 'additionalInfo',
       'gasPrices', 'questionsAndAnswers', 'updatesFromCustomers',
       'web

### Tratar columnas

In [6]:
# Usar pd.json_normalize para aplanar la columna 'location'
data_API = pd.concat([data_API, pd.json_normalize(data_API['location'])], axis=1)

# Eliminar la columna 'location' si ya no la necesitas
data_API = data_API.drop('location', axis=1)

# Me quedo con las columnas de interes
data_API=data_API[['url','searchString', 'state', 'lat', 'lng', 'reviews']]

# Quito los diccionarios que estan en listas
data_API = data_API.explode('reviews')

#Reseteo los indices
data_API=data_API.reset_index(drop=True)

# Utilizo json_normalize para expandir la columna 'reviews'
df_expanded  = pd.json_normalize(data_API['reviews'])

# Combina el DataFrame expandido con el DataFrame original
df_result = pd.concat([data_API, df_expanded], axis=1).drop(columns=['reviews'])

### Renombrar columnas

In [8]:
#Borramos nuevamente las columnas innecesarias
df_result=df_result[['url', 'reviewerId',  'stars', 'text', 'publishedAtDate',  'name', 'lat', 'lng', 'state']]

# Y las renombramos
df_result.rename(columns={'url':'business_url','reviewerId':'user_id',  'lat': 'latitude','lng': 'longitude', 'state':'state_name'}, inplace=True)

In [14]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12376 entries, 0 to 12375
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   business_url     12376 non-null  object 
 1   user_id          0 non-null      float64
 2   stars            12362 non-null  float64
 3   text             8137 non-null   object 
 4   publishedAtDate  12362 non-null  object 
 5   name             12362 non-null  object 
 6   latitude         12373 non-null  float64
 7   longitude        12373 non-null  float64
 8   state_name       12373 non-null  object 
dtypes: float64(4), object(5)
memory usage: 870.3+ KB


In [ ]:
import requests

# URL de la API
url = "https://api.apify.com/v2/datasets/W09Q96b7CANtZblZ2/items?token=apify_api_USgUEX5UbJ4Z0YMJoKF4S1MYt9mCmw4mmrzs&unwind=reviews&fields=placeId,postalCode,state,city,reviews&omit=textTranslated,publishAt,likesCount,reviewId,reviewUrl,reviewerUrl,reviewerPhotoUrl,reviewerNumberOfReviews,isLocalGuide,rating,reviewImageUrls,reviewContext,reviewDetailedRating"

# Realizar una solicitud GET a la API
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Los datos de la API están en formato JSON
    data = response.json()

    # Ahora puedes trabajar con los datos, por ejemplo, imprimirlos
    print(data)
else:
    # Si la solicitud no fue exitosa, imprimir el código de estado
    print(f"Error al obtener datos. Código de estado: {response.status_code}")


In [17]:
datos=pd.DataFrame(data=data)
datos.head(1)

# YELP

Extraigamos el Id de los locales para hacer el scrapp